In [1]:
from getpass import getpass
import os
import time

import pandas as pd
import requests

import kdbai_client as kdbai

from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFLoader
from langchain_community.vectorstores import KDBAI
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [2]:
KDBAI_ENDPOINT = input('KDB.AI endpoint: ')
KDBAI_API_KEY = getpass('KDB.AI API key: ')
os.environ['OPENAI_API_KEY'] = getpass('OpenAI API Key: ')

KDB.AI endpoint:  https://ui.qa.cld.kx.com/instance/pcnvlmi860
KDB.AI API key:  ········
OpenAI API Key:  ········


In [3]:
TEMP = 0.0
K = 3

In [4]:
print('Create KDB.AI session...')
session = kdbai.Session(endpoint=KDBAI_ENDPOINT, api_key=KDBAI_API_KEY)

Create KDB.AI session...


In [5]:
print('Create table "documents"...')
schema = {'columns': [{'name': 'id', 'pytype': 'str'},
                      {'name': 'text', 'pytype': 'bytes'},
                      {'name': 'embeddings',
                               'pytype': 'float32',
                               'vectorIndex': {'dims': 1536, 'metric': 'L2', 'type': 'hnsw'}},
                      {'name': 'tag', 'pytype': 'str'},
                      {'name': 'title', 'pytype': 'bytes'}]}
table = session.create_table('documents', schema)

Create table "documents"...


In [6]:
%%time
URL = 'https://www.conseil-constitutionnel.fr/node/3850/pdf'
PDF = 'Déclaration_des_droits_de_l_homme_et_du_citoyen.pdf'
open(PDF, 'wb').write(requests.get(URL).content)

CPU times: user 46.9 ms, sys: 6.82 ms, total: 53.8 ms
Wall time: 223 ms


562978

In [7]:
%%time
print('Read PDF...')
loader = PyPDFLoader(PDF)
pages = loader.load_and_split()
len(pages)

Read PDF...
CPU times: user 162 ms, sys: 12.1 ms, total: 174 ms
Wall time: 175 ms


3

In [8]:
%%time
print('Create Vector DB from PDF text...')
embeddings = OpenAIEmbeddings(model='text-embedding-ada-002')
texts = [p.page_content for p in pages]
metadata = pd.DataFrame(index=list(range(len(texts))))
metadata['tag'] = 'law'
metadata['title'] = 'Déclaration des Droits de l\'Homme et du Citoyen de 1789'.encode('utf-8')
vectordb = KDBAI(table, embeddings)
vectordb.add_texts(texts=texts, metadatas=metadata)

Create Vector DB from PDF text...
CPU times: user 226 ms, sys: 21.6 ms, total: 247 ms
Wall time: 2.46 s


['cb264350-5465-485c-876e-7618ec44b65e',
 '5bb3fe36-1447-4c7c-9c8b-faee35a74fe0',
 'c9e5a6d4-8c1d-4ba5-94aa-6c03e4a9f148']

In [9]:
%%time
print('Create LangChain pipeline...')
qabot = RetrievalQA.from_chain_type(chain_type='stuff',
                                    llm=ChatOpenAI(model='gpt-3.5-turbo-16k', temperature=TEMP), 
                                    retriever=vectordb.as_retriever(search_kwargs=dict(k=K)),
                                    return_source_documents=True)

Create LangChain pipeline...
CPU times: user 41.4 ms, sys: 4.51 ms, total: 45.9 ms
Wall time: 45.3 ms


In [10]:
%%time
Q = 'Summarize the document in English:'
print(f'\n\n{Q}\n')
print(qabot.invoke(dict(query=Q))['result'])



Summarize the document in English:

The document is the Declaration of the Rights of Man and of the Citizen of 1789. It was written by the representatives of the French people and aims to declare the natural, inalienable, and sacred rights of every individual. These rights include freedom, property, security, and resistance to oppression. The document emphasizes the importance of equality and the principle that sovereignty resides in the nation. It also highlights the role of law in protecting individual rights and ensuring the common good. The document asserts the right to freedom of thought, expression, and religion, as long as it does not disturb public order. It emphasizes the need for a public force to guarantee the rights of all citizens and the importance of a fair and equal distribution of public contributions. The document also recognizes the right of citizens to hold public officials accountable and states that any society without the guarantee of rights and separation of p

In [11]:
%%time
Q = 'Is it a fair law and why ?'
print(f'\n\n{Q}\n')
print(qabot.invoke(dict(query=Q))['result'])



Is it a fair law and why ?

As an AI language model, I don't have personal opinions. However, I can provide some analysis based on the given context. The text provided is an excerpt from the Declaration of the Rights of Man and of the Citizen of 1789, which is considered a foundational document in the history of human rights. It outlines the natural and inalienable rights of individuals, such as freedom, property, and resistance to oppression. 

Whether or not this law is considered fair is subjective and can vary depending on individual perspectives and societal norms. However, many people view the principles outlined in this declaration as fundamental to the concept of fairness and justice. It emphasizes the equality of all individuals, the importance of protecting individual rights, and the need for a just and accountable government. 

It is worth noting that this declaration was written in a specific historical and cultural context, and interpretations of fairness may differ acro

In [12]:
%%time
Q = 'What are the rights and duties of the man, the citizen and the society ?'
print(f'\n\n{Q}\n')
print(qabot.invoke(dict(query=Q))['result'])



What are the rights and duties of the man, the citizen and the society ?

According to the Declaration of the Rights of Man and of the Citizen of 1789, the rights and duties of man, citizen, and society are as follows:

Rights of Man:
1. Men are born and remain free and equal in rights. Social distinctions can only be based on common utility.
2. The purpose of political association is to protect the natural and imprescriptible rights of man, which include liberty, property, security, and resistance to oppression.
3. Sovereignty resides essentially in the nation, and no individual or group can exercise authority that does not come from the nation.
4. Freedom consists of being able to do anything that does not harm others. The exercise of natural rights is only limited by the law.
5. The law can only prohibit actions that are harmful to society. Anything that is not prohibited by law cannot be prevented, and no one can be compelled to do what the law does not require.
6. The law is the

In [13]:
%%time
Q = 'Is this law practical ?'
print(f'\n\n{Q}\n')
print(qabot.invoke(dict(query=Q))['result'])



Is this law practical ?

As an AI language model, I cannot provide personal opinions or subjective judgments on whether a law is practical or not. The provided text consists of articles from the French Declaration of the Rights of Man and of the Citizen of 1789, which is a foundational document in French constitutional law. It outlines fundamental rights and principles that are considered important in a democratic society. The practicality of a law is often a matter of interpretation and can vary depending on the context and specific circumstances. It is ultimately up to legal experts, lawmakers, and the judiciary to determine the practicality and application of a law.
CPU times: user 84.5 ms, sys: 5.42 ms, total: 89.9 ms
Wall time: 11.2 s


In [14]:
# Clean up KDB.AI "documents" table and index for similarity search
# so this notebook could be played again and again
session.table('documents').drop()

True